In [47]:
import pandas as pd
import numpy as np
import math
from pulp import *
from tabulate import tabulate

In [236]:
# ========= SET-UP =========== #
NUM_AREA = 3
R = 11.5 # Revenue
S = 1.2 # Multiplier

# ========== DATA ============= #

P1 = 0.98
P2 = 0.61
P3 = 0.18

x_ij = [[P1, P2, P3], 
        [P2, P1, P3], 
        [0,0,0]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.5, 999], 
        [4.5, 0, 999], 
        [6,6,0]] # c00, c01, c10, c11 # Move cars from i to j


d_i = [100,20,0]
n_0i = [5,44,9999]

# ========= INIT LP ============ #
prob = LpProblem("Model", LpMaximize)

m_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
y_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
n_1i = []
for i in range(NUM_AREA):
    n_1i.append(LpVariable("n_1%d"%(i), 0, None, LpInteger))
    for j in range(NUM_AREA):
        y_ij[i][j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)
        if i!=j:
            m_ij[i][j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

# ========= CONSTRUCT LP ============ #

# Objective
prob += sum([S*R*y_ij[i][j]*x_ij[i][j] - m_ij[i][j]*c_ij[i][j] for i in range(NUM_AREA) for j in range(NUM_AREA)])

# Subject To
for j in range(NUM_AREA):
    # n_1i
    prob += n_1i[j] == n_0i[j] + sum(m_ij[i][j] for i in range(NUM_AREA)) - sum(m_ij[j][i] for i in range(NUM_AREA)), "Equation Area: %d"%j
    # n_0i
    prob += n_0i[j] >= sum(m_ij[j][i] for i in range(NUM_AREA)), "Moving Constrain: %d"%j
    # y_ij
    prob += sum(y_ij[j][i] for i in range(NUM_AREA)) <= d_i[j] ,"Total Demand: %d"%j
    prob += sum(y_ij[i][j]*x_ij[i][j] for i in range(NUM_AREA)) <= n_1i[j] ,"Total Available: %d"%j
    

# ============== SOLVE ============= #
prob.solve()
print("\n ======== Optimal Variables ========= \n")
for v in prob.variables():
    print (v.name, "=", v.varValue)
print("Objective: " ,value(prob.objective))

print("\n ======== Model Details ========= \n")
results = []
for name, c in list(prob.constraints.items()):
    results.append([name,c,  c.pi,  c.slack])
print (tabulate(results, headers=['Constrain', 'Formula', "pi", "slack"]))


 ======== Optimal Variables ========= 

m_01 = 0.0
m_02 = 0.0
m_10 = 24.0
m_12 = 0.0
m_20 = 69.0
m_21 = 0.0
n_10 = 98.0
n_11 = 20.0
n_12 = 9930.0
y_00 = 100.0
y_01 = 0.0
y_02 = 0.0
y_10 = 0.0
y_11 = 20.0
y_12 = 0.0
y_20 = 0.0
y_21 = 0.0
y_22 = 0.0
Objective:  1100.8799999999999

 ======== Model Details ========= 

Constrain            Formula                                     pi           slack
-------------------  ----------------------------------------  ----  --------------
Equation_Area:_0     m_01 + m_02 - m_10 - m_20 + n_10 = 5        -0    -0
Moving_Constrain:_0  m_01 + m_02 <= 5                            -0     5
Total_Demand:_0      y_00 + y_01 + y_02 <= 100                   -0    -0
Total_Available:_0   -n_10 + 0.98*y_00 + 0.61*y_10 <= -0.0       -0    -4.26326e-14
Equation_Area:_1     -m_01 + m_10 + m_12 - m_21 + n_11 = 44      -0    -0
Moving_Constrain:_1  m_10 + m_12 <= 44                           -0    20
Total_Demand:_1      y_10 + y_11 + y_12 <= 20                

In [225]:
def optimizer (x_ij, c_ij, d_i, n_0i, R, NUM_AREA):
    prob = LpProblem("Model", LpMaximize)

    m_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
    y_ij = [[0]*NUM_AREA for _ in range(NUM_AREA)]
    n_1i = []
    for i in range(NUM_AREA):
        n_1i.append(LpVariable("n_1%d"%(i), 0, None, LpInteger))
        for j in range(NUM_AREA):
            y_ij[i][j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)
            if i!=j:
                m_ij[i][j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

    # ========= CONSTRUCT LP ============ #

    # Objective
    prob += sum([R*y_ij[i][j]*x_ij[i][j] - m_ij[i][j]*c_ij[i][j] for i in range(NUM_AREA) for j in range(NUM_AREA)])

    # Subject To
    for j in range(NUM_AREA):
        # n_1i
        prob += n_1i[j] == n_0i[j] + sum(m_ij[i][j] for i in range(NUM_AREA)) - sum(m_ij[j][i] for i in range(NUM_AREA)), "Equation Area: %d"%j
        # n_0i
        prob += n_0i[j] >= sum(m_ij[j][i] for i in range(NUM_AREA)), "Moving Constrain: %d"%j
        # y_ij
        prob += sum(y_ij[j][i] for i in range(NUM_AREA)) <= d_i[j] ,"Total Demand: %d"%j
        prob += sum(y_ij[i][j]*x_ij[i][j] for i in range(NUM_AREA)) <= n_1i[j] ,"Total Available: %d"%j


    # ============== SOLVE ============= #
    prob.solve()
    '''
    print("\n ======== Optimal Variables ========= \n")
    
    for v in prob.variables():
        print (v.name, "=", v.varValue)
    print("Objective: " ,value(prob.objective))

    print("\n ======== Model Details ========= \n")
    results = []
    for name, c in list(prob.constraints.items()):
        results.append([name,c,  c.pi,  c.slack])
    print (tabulate(results, headers=['Constrain', 'Formula', "pi", "slack"]))
    '''
    return prob.variables()[6:8]

In [227]:
# ========= SET-UP =========== #
NUM_AREA = 3
R = 11.5 # Revenue
S = 1.6 # Multiplier

# ========== DATA ============= #

# X_ij is very sensitive
x_ij = [[.985, 0.7, 0.2], 
        [0.7, .985, 0.2], 
        [0,0,0]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.5, 999], 
        [4.5, 0, 999], 
        [6,6,0]] # c00, c01, c10, c11 # Move cars from i to j

d_i = [100,20,0]
n_0i = [10,38,9999]

res  = optimizer (x_ij, c_ij, d_i, n_0i, R*S, NUM_AREA)
res

[n_10, n_11]

In [239]:
# ========= SET-UP =========== #
NUM_AREA = 3
R = 11.5 # Revenue
S = 1.2 # Multiplier

# ========== DATA ============= #

# X_ij is very sensitive
P1 = 0.98
P2 = 0.61
P3 = 0.18

x_ij = [[P1, P2, P3], 
        [P2, P1, 0.22], 
        [0,0,0]] # [[row]] - x00, x01, x10, x11, move cars from j to i

c_ij = [[0, 4.1, 999], 
        [4.1, 0, 999], 
        [6.5,5.1,0]] # c00, c01, c10, c11 # Move cars from i to j

#d_i = [100,20,0]
n_0i = [5,44,9999]

res = pd.DataFrame(columns = [ "n_10", "n_11"])

# out = optimizer (x_ij, c_ij, [100,20,0], n_0i, R*S, NUM_AREA)

for d1 in range(80,200):
    d_i = [d1, 20,0]
    out = optimizer (x_ij, c_ij, d_i, n_0i, R*S, NUM_AREA)
    
    res.loc[d1] = [out[0].varValue, out[1].varValue]

res


,n_10,n_11
80,77.0,21.0
81,78.0,21.0
82,79.0,21.0
83,80.0,21.0
84,81.0,21.0
85,82.0,21.0
86,83.0,21.0
87,84.0,21.0
88,85.0,21.0
89,86.0,21.0


In [240]:
res.to_csv("loop_di.csv")